In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import ode

# Assignment 2

## Question 1

In [ ]:
def deriv1(f: callable, x0: float, h: float) -> float:
    """Return the approximate derivative of function f at argument x0 calculated numerically 
    using steps of size h using method 1.
    
    """
    
    return (f(x0 + h) - f(x0)) / (h)

            
def deriv2(f: callable, x0: float, h: float) -> float:
    """Return the approximate derivative of function f at argument x0 calculated numerically 
    using steps of size h using method 2.
    
    """
    
    return (f(x0 + h) - f(x0 - h)) / (2 * h)

In [ ]:
x0 = 0.1
h = 0.4

print('Approximate derivative of sin(x) at x = 0.1 using method 1:', deriv1(np.sin, x0, h))
print('Approximate derivative of sin(x) at x = 0.1 using method 2:', deriv2(np.sin, x0, h))

# We know that the derivative of sin(x) at any x is cos(x)
print('Actual derivative of sin(x) at x = 0.1:', np.cos(x0))

In [ ]:
x0 = 0.1
step_size = np.linspace(0.01, 0.99, 99)

# Relative error is abs(numerical - analytic) / analytic
error1 = abs((deriv1(np.sin, x0, step_size) - np.cos(x0)) / np.cos(x0))
error2 = abs((deriv2(np.sin, x0, step_size) - np.cos(x0)) / np.cos(x0))

plt.plot(step_size, error1, color='red', label='Method 1', lw=2)
plt.plot(step_size, error2, color='royalblue', label='Method 2', lw=2)

plt.xlabel('Step size')
plt.ylabel('Relative error')
plt.title('Relative error of numerically evaluating the derivative of sin(x) at x = 0.1 \
with relation to step size')

plt.gcf().set_size_inches(15, 10)
plt.rc('font', size = 14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Same plot as above but in log scales

plt.loglog(step_size, error1, color='red', label='Method 1', lw=2)
plt.loglog(step_size, error2, color='royalblue', label='Method 2', lw=2)

plt.xlabel('Step size')
plt.ylabel('Relative error')
plt.title('Relative error of numerically evaluating the derivative of sin(x) at x = 0.1 \
in relation to step size')

plt.gcf().set_size_inches(15, 10)
plt.rc('font', size = 14)
plt.legend()
plt.grid()
#plt.savefig('Figure1.pdf')
plt.show()

The relation between the error and the step size is roughly linear in log scales.

The slope in the logarithmic plot represents the order of magnitude by which the step size affects the relative error.

In [ ]:
## Assuming the relation is truly linear, the slope can be determined:

slope1 = (np.log10(error1[-1]) - np.log10(error1[0])) / (np.log10(step_size[-1]) - np.log10(step_size[0]))
slope2 = (np.log10(error2[-1]) - np.log10(error2[0])) / (np.log10(step_size[-1]) - np.log10(step_size[0]))

print('Slope in log scale for method 1:', slope1)
print('Slope in log scale for method 2:', slope2)

In [ ]:
# Using numpy's polyfit to get a slope:

linearfit1 = np.polyfit(np.log10(step_size), np.log10(error1), 1)
linearfit2 = np.polyfit(np.log10(step_size), np.log10(error2), 1)

print('Slope in log scale for method 1:', linearfit1[0])
print('Slope in log scale for method 2:', linearfit2[0])

## Question 2

In [ ]:
# x and y are both an array of numbers from -2 to 2, but not including -2 and 2
x = np.linspace(-2, 2, 301)[1:-1]
y = np.linspace(-2, 2, 301)[1:-1]

z_dict = {}

In [ ]:
# Loop over every possible complex number c = x + iy
for a in x:
    for b in y:
        c = a + 1j * b
        
        # Create a list of z, starting with only the first value of z as 0
        z_list = [0]
        
        # Iterate and add the next value of z to the list, 
        # which depends on the previous value of z and on c
        for i in range(999):
            next_z = z_list[-1] ** 2 + c
            z_list.append(next_z)
        
        # Send all the iterations of z for a given c to the dictionary (with key c)
        z_dict[c] = z_list

In [ ]:
# Investigating the dictionary

a = x[129]
b = y[150]

c = a + 1j * b

print(z_dict[c])

In [ ]:
# Make an image where the ith row is the y-value and jth column is the x-value
# At first, set all values to zero
image1 = np.zeros((len(y), len(x)))

# Loop through all the keys c
for a in range(len(x)):
    for b in range(len(y)):
        c = x[a] + 1j * y[b]

        # If +inf or -inf is found in any iteration of z for a given c,
        # then give the image pixel for that c a value of one
        for z in z_dict[c]:
            if abs(z.real) == np.inf or abs(z.imag) == np.inf:
                image1[b][a] = 1

In [ ]:
plt.imshow(image1, cmap='plasma', origin='lower', 
           extent=[np.min(x), np.max(x), np.min(y), np.max(y)])

plt.axvline(x=0, color='black', lw=2)
plt.axhline(y=0, color='black', lw=2)

plt.xlabel('Real axis')
plt.ylabel('Imaginary axis')
plt.title('Locations in the complex plane where z diverges (shown in yellow)')

plt.gcf().set_size_inches(12, 12)
plt.rc('font', size = 16)
plt.grid()
#plt.savefig('Figure2.pdf')
plt.show()

In [ ]:
# Same strategy for initializing image2
image2 = np.zeros((len(y), len(x)))

for a in range(len(x)):
    for b in range(len(y)):
        c = x[a] + 1j * y[b]

        # This time, looking for the index of z where +inf or -inf is first found
        # Make a list of indices that have +inf or -inf
        indices = []
        
        # Append the index of infinite z to the list of indices
        for z in range(len(z_dict[c])):
            if abs(z_dict[c][z].real) == np.inf or abs(z_dict[c][z].imag) == np.inf:
                indices.append(z)
                
        # If an index was added to indices, 
        # give the image pixel for that c the value of the smallest index of z
        if indices != []:
            image2[b][a] = min(indices)
        
# This assumes that np.inf can appear in more than one iteration of z; 
# but, from inspection, np.inf seems to appear only in
# one part (real or imaginary) of the first iteration of diverging z 
# and all subsequent real and imaginary parts become np.nan.
# (But I have not looked at every entry in the dictionary.)

In [ ]:
img_mean = np.mean(image2)
img_std = np.std(image2)
img_max = img_mean + 3 * img_std

plt.imshow(image2, cmap='inferno', origin='lower', vmax=img_max, 
           extent=[np.min(x), np.max(x), np.min(y), np.max(y)])

plt.colorbar(label='Iteration number of divergence (no divergence at 0)')

plt.axvline(x=0, color='white', lw=1)
plt.axhline(y=0, color='white', lw=1)

plt.xlabel('Real axis')
plt.ylabel('Imaginary axis')
plt.title('Divergence of z in the complex plane')

plt.gcf().set_size_inches(15, 12)
plt.rc('font', size = 16)
plt.grid()
#plt.savefig('Figure3.pdf')
plt.show()

## Question 3

In [ ]:
# Set the initial condition vector
SIR_0 = np.array([999, 1, 0])

# Set initial time, endtime, and timesteps
t_0 = 0.0
t_end = 200
dt = 0.1

# Set constants N, beta, and gamma
N = 1000
beta = 3.0
gamma = 0.2

In [ ]:
# (S, I, R)' = (-beta S I / N, beta S I / N - gamma I, gamma I)

def SIRmodel(t: float, X_vector: np.ndarray) -> np.ndarray:
    """Return the rates of change in the numbers of suceptible individuals (dSdt), infected individuals (dIdt), 
    and recovered individuals (dRdt) given X_vector composed [S, I, R] and time t.
    
    """

    dSdt = -beta * X_vector[0] * X_vector[1] / N
    dIdt = (beta * X_vector[0] * X_vector[1] / N) - (gamma * X_vector[1])
    dRdt = gamma * X_vector[1]
    
    return np.array([dSdt, dIdt, dRdt])

In [ ]:
def solveode(f: callable, X_0: np.ndarray, t_0: float, t_end: float, dt: float) -> tuple:
    """Solve the ordinary differential equation described by f with initial conditions contained in X_0 
    and initial time t_0, endtime t_end, and timesteps dt.
    
    """
    
    solution = [X_0]
    times = [t_0]

    solver = ode(f).set_integrator('dopri5').set_initial_value(X_0, t_0)

    while solver.successful() and solver.t < t_end:
        times.append(solver.t + dt)
        solution.append(solver.integrate(solver.t + dt))

    return (np.array(solution), np.array(times))

In [ ]:
sol, t = solveode(SIRmodel, SIR_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible', lw=2)
plt.plot(t, sol[:, 1], color='red', label='Infected', lw=2)
plt.plot(t, sol[:, 2], color='royalblue', label='Removed (recovered)', lw=2)

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title('Disease evolution in a population of ' + str(N) + r' individuals with parameters $\beta$ = ' 
          + str(beta) + r' and $\gamma$ = ' + str(gamma))

plt.gcf().set_size_inches(15, 10)
plt.rc('font', size = 14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.subplot(2, 2, 1)

# Rapid spread, slow recovery
beta = 2.0
gamma = 0.1
sol, t = solveode(SIRmodel, SIR_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Removed (recovered)')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r' and $\gamma$ = ' + str(gamma))
plt.legend()
plt.grid()


plt.subplot(2, 2, 2)

# Rapid spread, rapid recovery
beta = 2.0
gamma = 0.8
sol, t = solveode(SIRmodel, SIR_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Removed (recovered)')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r' and $\gamma$ = ' + str(gamma))
plt.legend()
plt.grid()


plt.subplot(2, 2, 3)

# Slow spread, slow recovery
beta = 0.2
gamma = 0.1
sol, t = solveode(SIRmodel, SIR_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Removed (recovered)')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r' and $\gamma$ = ' + str(gamma))
plt.legend()
plt.grid()


plt.subplot(2, 2, 4)

# Slow spread, rapid recovey
beta = 1.0
gamma = 0.8
sol, t = solveode(SIRmodel, SIR_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Removed (recovered)')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r' and $\gamma$ = ' + str(gamma))
plt.legend()
plt.grid()


plt.suptitle('Disease evolution in a population of ' + str(N) + ' individuals')
plt.gcf().set_size_inches(16, 12)
plt.rc('font', size = 12)
#plt.savefig('Figure4.pdf')
plt.show()

In [ ]:
# Set the initial condition vector
SIRD_0 = np.array([999, 1, 0, 0])

# Set constants N, beta, and gamma
N = 1000
beta = 2.0
gamma = 0.6
delta = 0.1

In [ ]:
# (S, I, R, D)' = (-beta S I / N, beta S I / N - gamma I - delta I, gamma I, delta I)

def SIRDmodel(t: float, X_vector: np.ndarray) -> np.ndarray:
    """Return the rates of change in the numbers of suceptible individuals (dSdt), infected individuals (dIdt), 
    recovered individuals (dRdt), and deceased individuals (dDdt) given X_vector composed [S, I, R, D] 
    and time t.
    
    """

    dSdt = -beta * X_vector[0] * X_vector[1] / N
    dIdt = (beta * X_vector[0] * X_vector[1] / N) - (gamma * X_vector[1]) - (delta * X_vector[1])
    dRdt = gamma * X_vector[1]
    dDdt = delta * X_vector[1]
    
    return np.array([dSdt, dIdt, dRdt, dDdt])

In [ ]:
sol, t = solveode(SIRDmodel, SIRD_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible', lw=2)
plt.plot(t, sol[:, 1], color='red', label='Infected', lw=2)
plt.plot(t, sol[:, 2], color='royalblue', label='Recovered', lw=2)
plt.plot(t, sol[:, 3], color='black', label='Deceased', lw=2)

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title('Disease evolution in a population of ' + str(N) + r' individuals with parameters $\beta$ = ' 
          + str(beta) + r', $\gamma$ = ' + str(gamma) + r', and $\delta$ = ' + str(delta))

plt.gcf().set_size_inches(15, 10)
plt.rc('font', size = 14)
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.subplot(2, 2, 1)

# Rapid spread, slow recovery, high death rate
beta = 2.0
gamma = 0.1
delta = 0.05
sol, t = solveode(SIRDmodel, SIRD_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Recovered')
plt.plot(t, sol[:, 3], color='black', label='Deceased')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r', $\gamma$ = ' + str(gamma) + r', and $\delta$ = ' + str(delta))
plt.legend()
plt.grid()


plt.subplot(2, 2, 2)

# Rapid spread, rapid recovery, high death rate
beta = 2.0
gamma = 0.6
delta = 0.2
sol, t = solveode(SIRDmodel, SIRD_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Recovered')
plt.plot(t, sol[:, 3], color='black', label='Deceased')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r', $\gamma$ = ' + str(gamma) + r', and $\delta$ = ' + str(delta))
plt.legend()
plt.grid()


plt.subplot(2, 2, 3)

# Slow spread, slow recovery, low death rate
beta = 0.2
gamma = 0.1
delta = 0.01
sol, t = solveode(SIRDmodel, SIRD_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Recovered')
plt.plot(t, sol[:, 3], color='black', label='Deceased')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r', $\gamma$ = ' + str(gamma) + r', and $\delta$ = ' + str(delta))
plt.legend()
plt.grid()


plt.subplot(2, 2, 4)

# Slow spread, rapid recovey, low death rate
beta = 1.0
gamma = 0.8
delta = 0.05
sol, t = solveode(SIRDmodel, SIRD_0, t_0, t_end, dt)

plt.plot(t, sol[:, 0], color='green', label='Susceptible')
plt.plot(t, sol[:, 1], color='red', label='Infected')
plt.plot(t, sol[:, 2], color='royalblue', label='Recovered')
plt.plot(t, sol[:, 3], color='black', label='Deceased')

plt.xlabel('Time')
plt.ylabel('Number of indviduals')
plt.title(r'$\beta$ = ' + str(beta) + r', $\gamma$ = ' + str(gamma) + r', and $\delta$ = ' + str(delta))
plt.legend()
plt.grid()


plt.suptitle('Disease evolution in a population of ' + str(N) + ' individuals')
plt.gcf().set_size_inches(16, 12)
plt.rc('font', size = 12)
#plt.savefig('Figure5.pdf')
plt.show()

This model is similar to the SIR model, except the "removed" individuals are split between recovered and deceased.